<a href="https://colab.research.google.com/github/HoseinNekouei/fine_tuning_with_custom_dataset/blob/main/fine_tuning_imdb_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load Dataset**

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

# **Load the dataset and divide into training and testing set**

In [ ]:
from posixpath import join
from pathlib import Path

def read_imdb_split(split_dir):
  split_dir= Path(split_dir)
  texts=[]
  labels=[]

  for label_dir in ['neg', 'pos']:
    for text_file in (split_dir/label_dir).iterdir():

      texts.append(text_file.read_text())
      labels.append(0 if label_dir== 'neg' else 1)

  return texts, labels

train_texts, train_labels= read_imdb_split('/content/aclImdb/train')
test_texts, test_labels= read_imdb_split('/content/aclImdb/test')

shape=','.join(
    [f'train text length: {len(train_texts)}',
    f'train labels length: {len(train_labels)}',
    f'test text length: {len(test_texts)}',
    f'test labels length: {len(test_labels)}'
    ])

print(shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_texts, train_labels, test_size= 0.2, stratify= train_labels)

length= ','. join([
    f'X_train length: {len(X_train)}',
    f'X_val length: {len(X_val)}',
    f'y_train length: {len(y_train)}',
    f'y_val length: {len(y_val)}'
])

print(length)

# **Load Tokenizer**

In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer

In [ ]:
train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length= 256)
val_encodings= tokenizer(X_val, padding= True, truncation= True, max_length= 256)
test_encodings= tokenizer(test_texts, padding= True, truncation= True, max_length= 256)

# **Make Dataset and DataLoader**

In [ ]:
import torch

In [ ]:
class IMDBDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings= encodings
    self.labels= labels

  def __getitem__(self, idx):
    item= {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)


train_set= IMDBDataset(train_encodings, y_train)
val_set= IMDBDataset(val_encodings, y_val)
test_set= IMDBDataset(test_encodings, test_labels)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)

In [ ]:
next(iter(train_loader))

In [ ]:

from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()



optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()